PART 1A

In [1]:
# --- paths ---
base_dir = "/mnt/c/School files/Year 2 Modules/Learning with side projects/CNN projects/sports_classification"
train_dir = base_dir + "/train"
val_dir   = base_dir + "/valid"
test_dir  = base_dir + "/test"

# --- config ---
IMG_SIZE = (224, 224)
BATCH    = 16
SEED     = 1337

import tensorflow as tf

# Load datasets
train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    train_dir, image_size=IMG_SIZE, batch_size=BATCH, label_mode="int",
    shuffle=True, seed=SEED
)
val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    val_dir, image_size=IMG_SIZE, batch_size=BATCH, label_mode="int",
    shuffle=False
)
test_ds_raw = tf.keras.utils.image_dataset_from_directory(
    test_dir, image_size=IMG_SIZE, batch_size=BATCH, label_mode="int",
    shuffle=False
)

class_names = train_ds_raw.class_names
num_classes = len(class_names)
print("num_classes:", num_classes)

# Prefetch/shuffle
train_ds = train_ds_raw.shuffle(256).prefetch(1)
val_ds   = val_ds_raw.prefetch(1)
test_ds  = test_ds_raw.prefetch(1)


2025-09-30 18:08:23.633332: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-30 18:08:23.646691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759226903.658923    1715 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759226903.662505    1715 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759226903.673767    1715 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Found 13492 files belonging to 100 classes.


I0000 00:00:1759226927.165440    1715 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3623 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Found 500 files belonging to 100 classes.
Found 500 files belonging to 100 classes.
num_classes: 100


PART 2A

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

# Model with EfficientNetB0 backbone
inputs = keras.Input(shape=IMG_SIZE + (3,), name="image")
aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.05, 0.05),
], name="augment")

x = aug(inputs)
base = keras.applications.EfficientNetB0(
    include_top=False, weights="imagenet", input_tensor=x
)
base.trainable = False

x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax", dtype="float32")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=8,
                                           restore_best_weights=True, verbose=1)
reduce_lr  = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                               patience=3, min_lr=1e-6, verbose=1)

print("Stage 1: train head")
history1 = model.fit(train_ds, validation_data=val_ds, epochs=10,
                     callbacks=[reduce_lr, early_stop])

# Fine-tune top ~30%
for layer in base.layers[: int(0.7 * len(base.layers))]:
    layer.trainable = False
for layer in base.layers[int(0.7 * len(base.layers)):]:
    layer.trainable = True

model.compile(optimizer=keras.optimizers.Adam(1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

print("Stage 2: fine-tune")
history2 = model.fit(train_ds, validation_data=val_ds, epochs=10,
                     callbacks=[reduce_lr, early_stop])

print("Evaluating on test set…")
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.4f}")

# Optional: save your trained model
model.save("my_sports_model.h5")


Stage 1: train head
Epoch 1/10


I0000 00:00:1758787326.343516     937 cuda_dnn.cc:529] Loaded cuDNN version 90300


844/844 ━━━━━━━━━━━━━━━━━━━━ 99s 91ms/step - accuracy: 0.5091 - loss: 2.3468 - val_accuracy: 0.9200 - val_loss: 0.4013 - learning_rate: 0.0010
Epoch 2/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 65s 74ms/step - accuracy: 0.8712 - loss: 0.5442 - val_accuracy: 0.9300 - val_loss: 0.2650 - learning_rate: 0.0010
Epoch 3/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 67s 73ms/step - accuracy: 0.9051 - loss: 0.3761 - val_accuracy: 0.9480 - val_loss: 0.2191 - learning_rate: 0.0010
Epoch 4/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 68s 74ms/step - accuracy: 0.9201 - loss: 0.3046 - val_accuracy: 0.9400 - val_loss: 0.1926 - learning_rate: 0.0010
Epoch 5/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 69s 74ms/step - accuracy: 0.9285 - loss: 0.2530 - val_accuracy: 0.9420 - val_loss: 0.1805 - learning_rate: 0.0010
Epoch 6/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 69s 74ms/step - accuracy: 0.9397 - loss: 0.2148 - val_accuracy: 0.9540 - val_loss: 0.1596 - learning_rate: 0.0010
Epoch 7/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 67s 74ms/step - accuracy: 0.9468 - loss: 0.1981

E0000 00:00:1758788032.777090     809 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


844/844 ━━━━━━━━━━━━━━━━━━━━ 119s 118ms/step - accuracy: 0.8271 - loss: 0.6446 - val_accuracy: 0.9460 - val_loss: 0.1861 - learning_rate: 1.0000e-04
Epoch 2/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 101s 114ms/step - accuracy: 0.9421 - loss: 0.2044 - val_accuracy: 0.9480 - val_loss: 0.1536 - learning_rate: 1.0000e-04
Epoch 3/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 105s 119ms/step - accuracy: 0.9599 - loss: 0.1484 - val_accuracy: 0.9580 - val_loss: 0.1468 - learning_rate: 1.0000e-04
Epoch 4/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 106s 121ms/step - accuracy: 0.9711 - loss: 0.1116 - val_accuracy: 0.9580 - val_loss: 0.1269 - learning_rate: 1.0000e-04
Epoch 5/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 104s 119ms/step - accuracy: 0.9686 - loss: 0.1059 - val_accuracy: 0.9580 - val_loss: 0.1342 - learning_rate: 1.0000e-04
Epoch 6/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 105s 118ms/step - accuracy: 0.9793 - loss: 0.0810 - val_accuracy: 0.9560 - val_loss: 0.1256 - learning_rate: 1.0000e-04
Epoch 7/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 103s 115ms/

Test accuracy: 0.9760


In [5]:
from datetime import datetime

# timestamped filenames so you don’t overwrite older runs
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")

# recommended (new format)
model.save(f"sports_effnetb0_{stamp}.keras")

# optional legacy format
model.save(f"sports_effnetb0_{stamp}.h5")

print("✅ Model saved as:", f"sports_effnetb0_{stamp}.keras", "and", f"sports_effnetb0_{stamp}.h5")


✅ Model saved as: sports_effnetb0_20250925-163141.keras and sports_effnetb0_20250925-163141.h5


In [ ]:
# AUTHOR'S 98.40% accuracy model
from tensorflow import keras
from tensorflow.keras.layers import DepthwiseConv2D as TFDepthwiseConv2D

# Shim: ignore unexpected 'groups' argument
class DepthwiseConv2D_NoGroups(TFDepthwiseConv2D):
    def __init__(self, *args, groups=1, **kwargs):
        super().__init__(*args, **kwargs)

# Load pre-trained .h5 model
model = keras.models.load_model(
    "EfficientNetB0-100-(224 X 224)- 98.40.h5",
    custom_objects={"DepthwiseConv2D": DepthwiseConv2D_NoGroups},
    compile=False
)

# Compile for evaluation
model.compile(optimizer=keras.optimizers.Adam(1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Evaluate on YOUR test set
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy from pre-trained .h5: {test_acc:.4f}")


In [5]:
from tensorflow import keras
from tensorflow.keras.layers import DepthwiseConv2D as TFDepthwiseConv2D
import tensorflow as tf
from tensorflow.keras import layers

MODEL_PATH = "./sports_effnetb0_20250925-163141.h5"  # your file

# 1) Define a class with the exact saved name; accept groups, ignore it
class DepthwiseConv2D_NoGroups(TFDepthwiseConv2D):
    def __init__(self, *args, groups=1, **kwargs):
        super().__init__(*args, **kwargs)

# 2) Load using BOTH names in custom_objects (covers all cases)
model = keras.models.load_model(
    MODEL_PATH,
    custom_objects={
        "DepthwiseConv2D_NoGroups": DepthwiseConv2D_NoGroups,
        "DepthwiseConv2D": DepthwiseConv2D_NoGroups,
    },
    compile=False
)

# 3) Compile + (optional) quick evaluate
model.compile(optimizer=keras.optimizers.Adam(1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
print("Evaluate on test:")
print(model.evaluate(test_ds))

# 4) Save CLEAN copies (no custom layer needed next time)
#    Rebuild minimal same-topology EfficientNetB0 head, copy weights, save.
IMG_SIZE = (224, 224) if 'IMG_SIZE' not in globals() else IMG_SIZE
num_classes = 100 if 'num_classes' not in globals() else num_classes

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = inputs
base = tf.keras.applications.EfficientNetB0(include_top=False, weights=None, input_tensor=x)
y = layers.GlobalAveragePooling2D()(base.output)
y = layers.Dropout(0.3)(y)
outputs = layers.Dense(num_classes, activation="softmax", dtype="float32")(y)
clean = keras.Model(inputs, outputs)

clean.set_weights(model.get_weights())
clean.save("clean_effnetb0.keras")
clean.save("clean_effnetb0.h5")
print("Saved clean models: clean_effnetb0.keras / clean_effnetb0.h5 (these won’t need custom_objects)")


Evaluate on test:


I0000 00:00:1759227523.355163    1852 service.cc:152] XLA service 0x7162e00023d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759227523.355196    1852 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2025-09-30 18:18:43.488464: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1759227524.043995    1852 cuda_dnn.cc:529] Loaded cuDNN version 90300


 7/32 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9932 - loss: 3.3376

I0000 00:00:1759227531.619394    1852 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 251ms/step - accuracy: 0.9850 - loss: 3.3680
[3.3856730461120605, 0.9800000190734863]


ValueError: You called `set_weights(weights)` on layer 'functional' with a weight list of length 367, but the layer was expecting 314 weights.

In [6]:
import os, numpy as np, pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, top_k_accuracy_score
import matplotlib.pyplot as plt

OUT = "analysis_outputs"; os.makedirs(OUT, exist_ok=True)

# 1) collect y_true, y_probs, y_pred
y_true, y_probs = [], []
for xb, yb in test_ds:
    probs = model.predict(xb, verbose=0)
    y_probs.append(probs)
    y_true.append(yb.numpy())
y_true = np.concatenate(y_true)
y_probs = np.concatenate(y_probs)
y_pred = y_probs.argmax(axis=1)

# 2) confusion matrix (counts + normalized) + plot
cm = confusion_matrix(y_true, y_pred, labels=range(num_classes))
cm_norm = cm.astype(float) / (cm.sum(axis=1, keepdims=True) + 1e-9)

pd.DataFrame(cm, index=class_names, columns=class_names).to_csv(f"{OUT}/confusion_matrix_counts.csv")
pd.DataFrame(cm_norm, index=class_names, columns=class_names).to_csv(f"{OUT}/confusion_matrix_normalized.csv")

plt.figure(figsize=(16,16), dpi=200)
plt.imshow(cm_norm, interpolation="nearest")
plt.title("Confusion Matrix (Normalized)")
plt.colorbar(fraction=0.046, pad=0.04)
N = max(1, len(class_names)//20)
ticks = list(range(0, len(class_names), N))
plt.xticks(ticks, [class_names[i][:12] for i in ticks], rotation=90)
plt.yticks(ticks, [class_names[i][:12] for i in ticks])
plt.tight_layout()
plt.savefig(f"{OUT}/confusion_matrix_normalized.png"); plt.close()

# 3) per-class precision / recall / f1
rep = classification_report(y_true, y_pred, target_names=class_names, output_dict=True, zero_division=0)
pd.DataFrame(rep).to_csv(f"{OUT}/classification_report.csv")

# 4) optional: top-5 accuracy for README
try:
    top5 = top_k_accuracy_score(y_true, y_probs, k=5, labels=range(num_classes))
    print(f"Top-1: {(y_pred==y_true).mean():.4f} | Top-5: {top5:.4f}")
except Exception:
    print(f"Top-1: {(y_pred==y_true).mean():.4f} (top-5 skipped)")

print("saved:",
      f"{OUT}/confusion_matrix_counts.csv,",
      f"{OUT}/confusion_matrix_normalized.csv,",
      f"{OUT}/confusion_matrix_normalized.png,",
      f"{OUT}/classification_report.csv")


2025-09-30 18:44:47.928495: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Top-1: 0.9800 | Top-5: 1.0000
saved: analysis_outputs/confusion_matrix_counts.csv, analysis_outputs/confusion_matrix_normalized.csv, analysis_outputs/confusion_matrix_normalized.png, analysis_outputs/classification_report.csv
